In [ ]:
import io
import requests

from PIL import Image as im

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

In [ ]:
import pandas as pd

pdf = pd.read_parquet('../../ftransfer_ztf_2024-02-01_689626')


In [ ]:
pdf

In [ ]:
Id = pdf['objectId'][9888]

id_plus_repete = pdf['objectId'].value_counts().idxmax()

# We select data based on their shared ID, (here opting for the most frequently occurring alert).
pdf_selectionne = pdf.loc[pdf['objectId'] == id_plus_repete]



In [ ]:
candidate_df = pdf_selectionne['candidate'].apply(pd.Series)

#  index du candidat avec le plus grand 'jd'
index_max_jd = candidate_df['jd'].idxmax()

# Sélectionner le candidat correspondant
pdf_selectionne = pdf_selectionne.loc[index_max_jd]


In [ ]:
# Selecting the final alert ( if the data is ordered).
#pdf_selectionne = pdf_selectionne.iloc[-1]

In [ ]:
pdf_selectionne['objectId']

In [ ]:
pdf_selectionne_cand = pdf_selectionne['prv_candidates'] 

In [ ]:
#  'candidate' the actual value 
keys = pdf_selectionne_cand[0].keys()
actual_cand = {key: pdf_selectionne['candidate'][key] for key in keys if key in pdf_selectionne['candidate']}


In [ ]:
liste_dicts = list(pdf_selectionne_cand)
liste_dicts.append(actual_cand)
df = pd.DataFrame(liste_dicts)

In [ ]:

mjd = df['jd'].apply(lambda x: x - 2400000.5)

fig = plt.figure(figsize=(25, 15))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}

# valid values 
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)
# Upper limit values
maskUpper = pd.isna(df['magpsf'])
#bad quality values 
maskBadquality = ~maskValid & ~maskUpper


for filt in np.unique(df['fid']):
    maskFilt = df['fid'] == filt

    plt.errorbar(
        df[maskValid & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt]['magpsf'],
        df[maskValid & maskFilt]['sigmapsf'],
        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )

    plt.plot(
        df[maskUpper & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskUpper & maskFilt]['diffmaglim'],
        ls='', marker='v', color=colordic[filt], markerfacecolor='none'
    )
    

    plt.errorbar(
        df[maskBadquality & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskBadquality & maskFilt]['magpsf'],
        df[maskBadquality & maskFilt]['sigmapsf'],
        ls='', marker='^', color=colordic[filt]
    )

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');


In [ ]:

 

fig = plt.figure(figsize=(15, 5))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}
    
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)

    
#t or 1 => candidate is from positive (sci minus ref) subtraction;
#f or 0 => candidate is from negative (ref minus sci) subtraction"
maskneg = (df['isdiffpos'] == 't') | (df['isdiffpos'] == 1)
maskpos = (df['isdiffpos'] == 'f') | (df['isdiffpos'] == 0)

for filt in np.unique(df['fid']):
    maskFilt = df['fid'] == filt

    # candidates from negative 

    plt.errorbar(
        df[maskValid & maskFilt & maskneg ]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt & maskneg ]['magpsf'],
        df[maskValid & maskFilt & maskneg ]['sigmapsf'],
        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )
    
    # candidates from positive 
    plt.errorbar(
        df[maskValid & maskFilt &  maskpos ]['jd'].apply(lambda x: x - 2400000.5),
        df[maskValid & maskFilt &  maskpos ]['magpsf'],
        df[maskValid & maskFilt &  maskpos ]['sigmapsf'],
        ls = '', marker='^', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )
    

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');



In [ ]:
# from fink_science.conversion import dc_mag
from fink_utils.photometry.conversion import dc_mag
from fink_utils.photometry.utils import is_source_behind

# Take only valid measurements
maskValid = (df['rb'] > 0.55) & (df['nbad'] == 0)
df_valid = df[maskValid].sort_values('jd')#, ascending=False)

isSource = is_source_behind(
    df_valid['distnr'].values[0]
)

if isSource:
    print('It looks like there is a source behind. Lets compute the DC magnitude instead.')
    
    # Use DC magnitude instead of difference mag
    mag_dc, err_dc = np.transpose(
        [
            dc_mag(*args) for args in zip(
                df_valid['magpsf'].astype(float).values,
                df_valid['sigmapsf'].astype(float).values,
                df_valid['magnr'].astype(float).values,
                df_valid['sigmagnr'].astype(float).values,
                df_valid['isdiffpos'].values
            )
        ]
    )
    
    df_valid['mag_dc'] = mag_dc
    df_valid['err_dc'] = err_dc
else:
    print('No source found -- keeping PSF fit magnitude')
    df_valid['mag_dc'] = df_valid['magpsf']
    df_valid['err_dc'] = df_valid['sigmapsf']



In [ ]:
fig = plt.figure(figsize=(15, 5))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}

for filt in np.unique(df_valid['fid']):
    maskFilt = df_valid['fid'] == filt

    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['magpsf'],
        df_valid[maskFilt]['sigmapsf'],
        ls = '', marker='x', 
        color=colordic[filt], 
        label='{} band (PSF-fit)'.format(filtdic[filt]),
    )
    
    
    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['mag_dc'],
        df_valid[maskFilt]['err_dc'],
        ls = '', marker='o', 
        color=colordic[filt], 
        label='{} band (DC)'.format(filtdic[filt]),
    )
    #To show if there is a variance in the reference
    plt.errorbar(
        df_valid[maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df_valid[maskFilt]['magnr'],
        ls = '--', 
        color=colordic[filt], 
        label='{} ref (DC)'.format(filtdic[filt]),
    )
    

ref_r = df_valid['magnr']*(df_valid['fid']==2)

ref_r = (ref_r[ref_r != 0]).mean()

ref_g = df_valid['magnr']*(df_valid['fid']==1)

ref_g = (ref_g[ref_g != 0]).mean()


print(ref_r, ref_g)
#  Average Values (Used in Fink)
#plt.axhline(y=ref_r, color=colordic[2], linestyle='--')
#plt.axhline(y=ref_g, color=colordic[1], linestyle='--')
plt.gca().invert_yaxis()
plt.legend()
plt.title('Comparison of PSF-fit and DC magnitudes')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Magnitude');

In [ ]:
fig = plt.figure(figsize=(15, 7))

from fink_utils.photometry.conversion import apparent_flux


dc_flux, dc_sigflux = np.transpose(
        [
            apparent_flux(*args, jansky=True) for args in zip(
                df_valid['magpsf'].astype(float).values,
                df_valid['sigmapsf'].astype(float).values,
                df_valid['magnr'].astype(float).values,
                df_valid['sigmagnr'].astype(float).values,
                df_valid['isdiffpos'].values
            )
        ]
)

df_valid['dc_flux'] = dc_flux
df_valid['dc_sigflux'] = dc_sigflux


for filt in np.unique(df['fid']):
    mask = df_valid['fid'] == filt
    sub = df_valid[mask]
    plt.errorbar(
        sub['jd'].apply(lambda x: x - 2400000.5),
        sub['dc_flux']*1e3,#*1e7/3, 
        sub['dc_sigflux']*1e3,#*1e7/3, 
        ls='', 
        marker='o',
        label=filt
    )
plt.legend()

plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Apparent DC flux (millijanksy)');


# Data missing 


In [ ]:
# Take only valid measurements
maskUpper = pd.isna(df['magpsf'])

df_Upper = df[maskUpper].sort_values('jd')#, ascending=False)


In [ ]:
sigmnr_r = df_valid['sigmagnr']*(df_valid['fid']==2)

sigmnr_r = (sigmnr_r[sigmnr_r != 0]).mean()

sigmnr_g = df_valid['sigmagnr']*(df_valid['fid']==1)

sigmnr_g = (sigmnr_g[sigmnr_g != 0]).mean()


In [ ]:
print(sigmnr_r,sigmnr_g)

In [ ]:
df_valid[['sigmagnr','magnr']]

In [ ]:
import numpy as np
from typing import Tuple

def apparent_flux_Upper(
    diffmaglim: float,
    fid: int,
    ref_r: float,
    ref_g: float,
    sigmnr_r: float,
    sigmnr_g: float,
    jansky: bool = True

) -> Tuple[float, float]:
    """
    
    Parameters
    ---------

    Returns
    --------
    dc_flux: float
        Apparent flux
    dc_sigflux: float
        Error on apparent flux
    """
    
    #difference_flux = 10 ** (-0.4 * magpsf)
    #difference_sigflux = (sigmapsf / 1.0857) * difference_flux

    #ref_flux = 10 ** (-0.4 * magnr)
    
    if (fid == 1):
        mnr = ref_g
        sigmnr = sigmnr_g
    if (fid == 2):
        mnr = ref_r
        sigmnr = sigmnr_r

    dc_flux = 10 ** (-0.4 * mnr)
    
    sig_c = (10 ** (-0.4 * diffmaglim))/np.sqrt(3)
    sig_flux_nr = np.log(10) * 0.4 * (10 ** (-0.4 * mnr)) *sigmnr # dc_flux ! 
    
    dc_sigflux = np.sqrt(sig_c**2 + sig_flux_nr**2)
    print( sig_flux_nr*3631)

    if jansky:
        dc_flux *= 3631
        dc_sigflux *= 3631

    return dc_flux, sig_c*3631#dc_sigflux


In [ ]:

dc_flux, dc_sigflux = np.transpose(
        [
            apparent_flux_Upper(*args, ref_r, ref_g, sigmnr_r, sigmnr_g, jansky=True) for args in zip(
                df_Upper['diffmaglim'].astype(float).values,
                df_Upper['fid'].astype(int).values,

            )
        ]
)

df_Upper['dc_flux'] = dc_flux
df_Upper['dc_sigflux'] = dc_sigflux


In [ ]:
df_valid['sigmapsf']

In [ ]:
dc_sigflux

In [ ]:
fig = plt.figure(figsize=(15, 7))


for filt in np.unique(df['fid']):
    mask = df_valid['fid'] == filt
    sub = df_valid[mask]
    plt.errorbar(
        sub['jd'].apply(lambda x: x - 2400000.5),
        sub['dc_flux']*1e3, 
        sub['dc_sigflux']*1e3,
        ls='', 
        marker='o',
        color=colordic[filt], 

        label=f"{filt} valid"
    )
    
    mask2 = df_Upper['fid'] == filt

    plt.errorbar(
        df_Upper[mask2]['jd'].apply(lambda x: x - 2400000.5),
        df_Upper[mask2]['dc_flux']*1e3,
        df_Upper[mask2]['dc_sigflux']*1e3,
        ls='', 
        marker='.',
        color=colordic[filt], 

        label=f"{filt} Upperlim"
    )
    
    
    
plt.legend()

plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Apparent DC flux (millijanksy)');

In [ ]:
df_valid[df_valid['fid']==2]['dc_sigflux']

In [ ]:
df_Upper['magnr']

In [ ]:
df_valid['dc_sigflux']

# Data by days 

In [ ]:
df_valid['mjd'] = df_valid['jd'].apply(lambda x: x - 2400000.5)

In [ ]:
# Convert 'mjd' to integer to remove fractional part
df_valid['mjd'] = df_valid['mjd'].astype(int)
df2 = df_valid.groupby(['mjd','fid'])

In [ ]:
df2.size()

In [ ]:
counts_per_day = df2.count()
"""#sum_per_day = df2.sum()

median_per_day = df2.median()
min_per_day = df2.min()
max_per_day = df2.max()
std_per_day = df2.std()
variance_per_day = df2.var()
first_per_day = df2.first()
last_per_day = df2.last()"""
counts_per_day

In [ ]:
df2['sigmapsf']

In [ ]:
# Assuming df2 contains the DataFrame grouped by 'mjd'
# Calculate the average of 'magpsf' for each day
df_mod = df2['magpsf'].mean().reset_index()

# Resetting index will convert the 'mjd' index to a column and reset the row index


In [ ]:
df2['sigmapsf'].sum()

In [ ]:
def custom_calculation(group):
    if len(group) ==1 :
        return group['sigmapsf']

    sum_value = 1/ group['sigmapsf'].sum()  
    wi = 1/group['sigmapsf']**2
    
    
    x_bar = (group['magpsf']*wi).sum() / (wi).sum()
    
    x_2 = ((group['magpsf'] - x_bar)**2 *wi).sum()
    
    sigma_x = np.sqrt(1/ wi.sum()) * np.sqrt(x_2/ (len(group)-1))
    return pd.Series(sigma_x, index=[group.index[0]])
    return sigma_x


In [ ]:
result = df2.apply(custom_calculation)

result = result.reset_index(drop=True)

df_mod['sigmapsf'] = result

In [ ]:
df_mod

In [ ]:
df_valid[['mjd','magpsf','sigmapsf']] 

In [ ]:

fig = plt.figure(figsize=(25, 15))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}



for filt in np.unique(df['fid']):
    maskFilt = df_mod['fid'] == filt

    plt.errorbar(
        df_mod[maskFilt]['mjd'],
        df_mod[maskFilt]['magpsf'],
        df_mod[maskFilt]['sigmapsf'],

        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )

    """plt.plot(
        df[maskUpper & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskUpper & maskFilt]['diffmaglim'],
        ls='', marker='v', color=colordic[filt], markerfacecolor='none'
    )
    

    plt.errorbar(
        df[maskBadquality & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskBadquality & maskFilt]['magpsf'],
        df[maskBadquality & maskFilt]['sigmapsf'],
        ls='', marker='^', color=colordic[filt]
    )"""

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');


In [ ]:
fig = plt.figure(figsize=(25, 15))

colordic = {1: 'C0', 2: 'C1'}
filtdic = {1: 'g', 2: 'r'}



for filt in np.unique(df['fid']):
    maskFilt = df_valid['fid'] == filt

    plt.errorbar(
        df_valid[maskFilt]['mjd'],
        df_valid[maskFilt]['magpsf'],
        df_valid[maskFilt]['sigmapsf'],

        ls = '', marker='o', color=colordic[filt], label='{} band'.format(filtdic[filt])
    )

    """plt.plot(
        df[maskUpper & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskUpper & maskFilt]['diffmaglim'],
        ls='', marker='v', color=colordic[filt], markerfacecolor='none'
    )
    

    plt.errorbar(
        df[maskBadquality & maskFilt]['jd'].apply(lambda x: x - 2400000.5),
        df[maskBadquality & maskFilt]['magpsf'],
        df[maskBadquality & maskFilt]['sigmapsf'],
        ls='', marker='^', color=colordic[filt]
    )"""

plt.ylim(12, 22)
plt.gca().invert_yaxis()
plt.legend()
plt.title('Difference image PSF-fit magnitude')
plt.xlabel('Modified Julian Date [UTC]')
plt.ylabel('Difference Magnitude');


In [ ]:
id_plus_repete